In [1]:
from sklearn.pipeline import Pipeline
from functions import *
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from functions import * 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn
from matplotlib import pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import random
from sklearn.model_selection import cross_validate as cvt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier as dtc
from copy import deepcopy as dp

def tokenize(text):
    return text.split(' ')

In [2]:
test, train = load_pkl(r'C:\Users\HEndo\Documents\GitHub\zemi\cleaned_corpus\base_cleaned.pkl')
test_a, train_a = load_pkl(r'C:\Users\HEndo\Documents\GitHub\zemi\cleaned_corpus\org_answer_data.pkl')
train_X = dp(train)
train_Y = dp(train_a)

In [4]:
import optuna

Using TensorFlow backend.


In [5]:
def objective(trial):
    params = {
    '_min': trial.suggest_loguniform('_min', 1e-3, 1e-1),
    '_max': trial.suggest_discrete_uniform('_max',0.5, 0.9, 0.1),
    'ngram': trial.suggest_categorical('ngram',[(1,1), (1,2)]),
    'mx_depth': trial.suggest_discrete_uniform('mx_depth', 5, 150, 5),
    'mn_sp': trial.suggest_uniform('mn_sp', 0.1, 0.3),
    'mx_feat': trial.suggest_uniform('mx_feat', 0.1, 1.0)
    }
    sw = stopwords.words('english') + ['hes']
    data = list(zip(train_X, train_Y))
    random.shuffle(data)
    ff_train, ff_train_a = zip(*data)
    ff_train = [' '.join(text) for text in ff_train]
    vect = TfidfVectorizer(stop_words=sw,tokenizer=tokenize, 
                          min_df= params['_min'], max_df=params['_max'], 
                          ngram_range=params['ngram'])
    tfidf = vect.fit_transform(ff_train)
    tree = dtc(max_depth=params['mx_depth'], 
               min_samples_split= params['mn_sp'],
               min_samples_leaf= params['mn_lf'],
               max_features= params['mx_feat']
              )

    scores = cvt(tree, tfidf, ff_train_a, cv = 4, return_train_score= True, scoring = 'f1_macro', )
    test_s = scores['test_score'].mean()
    train_s = scores['train_score'].mean()
    f_score = 1.0 - float(test_s) + float(abs(train_s-test_s))/2.0
    return f_score

In [5]:
import warnings
warnings.filterwarnings('ignore')
optuna.logging.enable_default_handler()


In [7]:
study = optuna.create_study()
study.optimize(objective, n_trials=1000, n_jobs = -1)


[I 2019-02-25 17:12:25,064] Finished trial 100 / 1000. Current best value is 0.31496667696863734 with parameters: {'_min': 0.007989289170602388, '_max': 0.5, 'ngram': (1, 1), 'mx_depth': 30.0, 'mn_sp': 0.11616103717130016, 'mn_lf': 1, 'mx_feat': 0.3475732714087273}.
[I 2019-02-25 17:13:00,109] Finished trial 200 / 1000. Current best value is 0.31090697421803454 with parameters: {'_min': 0.004980417349302144, '_max': 0.7000000000000001, 'ngram': (1, 2), 'mx_depth': 35.0, 'mn_sp': 0.1201537750361901, 'mn_lf': 1, 'mx_feat': 0.3031566249684499}.
[I 2019-02-25 17:13:01,283] Finished trial 200 / 1000. Current best value is 0.31090697421803454 with parameters: {'_min': 0.004980417349302144, '_max': 0.7000000000000001, 'ngram': (1, 2), 'mx_depth': 35.0, 'mn_sp': 0.1201537750361901, 'mn_lf': 1, 'mx_feat': 0.3031566249684499}.
[I 2019-02-25 17:14:02,373] Finished trial 300 / 1000. Current best value is 0.3054546441557692 with parameters: {'_min': 0.0032157386496844517, '_max': 0.7000000000000001

In [8]:
print(study.best_params)
print(study.best_value)
print(study.best_trial)

{'_min': 0.003941570792418628, '_max': 0.7000000000000001, 'ngram': (1, 2), 'mx_depth': 140.0, 'mn_sp': 0.17573596009797782, 'mn_lf': 1, 'mx_feat': 0.8376154434188189}
0.3009616598197175
FrozenTrial(trial_id=581, state=<TrialState.COMPLETE: 1>, value=0.3009616598197175, datetime_start=datetime.datetime(2019, 2, 25, 17, 17, 24, 528965), datetime_complete=datetime.datetime(2019, 2, 25, 17, 17, 28, 309264), params={'_min': 0.003941570792418628, '_max': 0.7000000000000001, 'ngram': (1, 2), 'mx_depth': 140.0, 'mn_sp': 0.17573596009797782, 'mn_lf': 1, 'mx_feat': 0.8376154434188189}, user_attrs={}, system_attrs={}, intermediate_values={}, params_in_internal_repr={'_min': 0.003941570792418628, '_max': 0.7000000000000001, 'ngram': 1, 'mx_depth': 140.0, 'mn_sp': 0.17573596009797782, 'mn_lf': 0, 'mx_feat': 0.8376154434188189})


In [9]:
def objective2(trial):
    params = {
    '_min': trial.suggest_uniform('_min', 1e-3, 8*1e-3),
    'mx_depth': trial.suggest_discrete_uniform('mx_depth', 10, 500, 10),
    }
    sw = stopwords.words('english') + ['hes']
    data = list(zip(train_X, train_Y))
    random.shuffle(data)
    ff_train, ff_train_a = zip(*data)
    ff_train = [' '.join(text) for text in ff_train]
    vect = TfidfVectorizer(stop_words=sw,tokenizer=tokenize, 
                          min_df= params['_min'], max_df=0.75, 
                          ngram_range=(1,2))
    tfidf = vect.fit_transform(ff_train)
    tree = dtc(max_depth=abs(params['mx_depth']), 
               min_samples_split= 0.18,
               max_features= 0.85
              )

    scores = cvt(tree, tfidf, ff_train_a, cv = 4, return_train_score= True, scoring = 'f1_macro', )
    test_s = scores['test_score'].mean()
    train_s = scores['train_score'].mean()
    f_score = 1.0 - float(test_s) + float(abs(train_s-test_s))/2.0
    
    return f_score

In [10]:
study2 = optuna.create_study()
study2.optimize(objective2, n_trials=300, n_jobs = 1, verb_pace = 100)


[I 2019-02-27 09:56:14,026] Finished trial 100 / 300. Current best value is 0.2450829739016842 with parameters: {'_min': 0.0032464550033867497, 'mx_depth': 30.0}.
[I 2019-02-27 09:57:55,687] Finished trial 200 / 300. Current best value is 0.24288514726194826 with parameters: {'_min': 0.004641281633399373, 'mx_depth': 30.0}.
[I 2019-02-27 09:59:44,619] Finished trial 300 / 300. Current best value is 0.24288514726194826 with parameters: {'_min': 0.004641281633399373, 'mx_depth': 30.0}.


In [16]:
data = load_pkl('tuned_parameters/decision_tree.pkl')
save_pkl('tuned_parameters/decision_tree.pkl', list(data).append(study2))

In [18]:
def objective3(trial):
    params = {
    'mx_depth': trial.suggest_discrete_uniform('mx_depth', 10, 500, 10),
    }
    sw = stopwords.words('english') + ['hes']
    data = list(zip(train_X, train_Y))
    random.shuffle(data)
    ff_train, ff_train_a = zip(*data)
    ff_train = [' '.join(text) for text in ff_train]
    vect = TfidfVectorizer(stop_words=sw,tokenizer=tokenize, 
                          min_df= 0.0045, max_df=0.75, 
                          ngram_range=(1,2))
    tfidf = vect.fit_transform(ff_train)
    tree = dtc(max_depth=abs(params['mx_depth']), 
               min_samples_split= 0.18,
               max_features= 0.85
              )

    scores = cvt(tree, tfidf, ff_train_a, cv = 4, return_train_score= True, scoring = 'f1_macro', )
    test_s = scores['test_score'].mean()
    train_s = scores['train_score'].mean()
    f_score = 1.0 - float(test_s) + float(abs(train_s-test_s))/2.0
    
    return f_score

In [19]:
study3 = optuna.create_study()
study3.optimize(objective3, n_trials=300, n_jobs = 1, verb_pace = 50)


[I 2019-02-27 10:10:36,656] Finished trial 50 / 300. Current best value is 0.2529064872768976 with parameters: {'mx_depth': 30.0}.
[I 2019-02-27 10:11:23,935] Finished trial 100 / 300. Current best value is 0.24894499440492773 with parameters: {'mx_depth': 50.0}.
[I 2019-02-27 10:12:12,156] Finished trial 150 / 300. Current best value is 0.2392451300984194 with parameters: {'mx_depth': 20.0}.
[I 2019-02-27 10:12:59,031] Finished trial 200 / 300. Current best value is 0.2392451300984194 with parameters: {'mx_depth': 20.0}.
[I 2019-02-27 10:13:49,823] Finished trial 250 / 300. Current best value is 0.2392451300984194 with parameters: {'mx_depth': 20.0}.
[I 2019-02-27 10:14:39,420] Finished trial 300 / 300. Current best value is 0.2392451300984194 with parameters: {'mx_depth': 20.0}.


In [25]:
data = load_pkl('tuned_parameters/decision_tree.pkl')
save_pkl('tuned_parameters/decison_tree2.pkl', [study3])

In [26]:
def objective_class_weight(trial):
    params = {
    '_min': trial.suggest_uniform('_min', 0.001, 0.005),
    'mx_depth': trial.suggest_discrete_uniform('mx_depth', 10, 100, 10),
    }
    sw = stopwords.words('english') + ['hes']
    data = list(zip(train_X, train_Y))
    random.shuffle(data)
    ff_train, ff_train_a = zip(*data)
    ff_train = [' '.join(text) for text in ff_train]
    vect = TfidfVectorizer(stop_words=sw,tokenizer=tokenize, 
                          min_df= params['_min'], max_df=0.75, 
                          ngram_range=(1,2))
    tfidf = vect.fit_transform(ff_train)
    tree = dtc(max_depth=abs(params['mx_depth']), 
               min_samples_split= 0.18,
               max_features= 0.85
              )

    scores = cvt(tree, tfidf, ff_train_a, cv = 4, return_train_score= True, scoring = 'f1_macro', )
    test_s = scores['test_score'].mean()
    train_s = scores['train_score'].mean()
    f_score = 1.0 - float(test_s) + float(abs(train_s-test_s))/2.0
    
    return f_score

In [27]:
study4 = optuna.create_study()
study4.optimize(objective_class_weight, n_trials=300, n_jobs = 1, verb_pace = 50)

[I 2019-02-27 11:06:50,153] Finished trial 50 / 300. Current best value is 0.24472081719692385 with parameters: {'_min': 0.004786872775332392, 'mx_depth': 30.0}.
[I 2019-02-27 11:07:37,071] Finished trial 100 / 300. Current best value is 0.24472081719692385 with parameters: {'_min': 0.004786872775332392, 'mx_depth': 30.0}.
[I 2019-02-27 11:08:23,911] Finished trial 150 / 300. Current best value is 0.24345291868434948 with parameters: {'_min': 0.0023715426047840713, 'mx_depth': 20.0}.
[I 2019-02-27 11:09:08,879] Finished trial 200 / 300. Current best value is 0.24269605823130108 with parameters: {'_min': 0.004028593139679666, 'mx_depth': 20.0}.
[I 2019-02-27 11:09:54,552] Finished trial 250 / 300. Current best value is 0.24269605823130108 with parameters: {'_min': 0.004028593139679666, 'mx_depth': 20.0}.
[I 2019-02-27 11:10:39,431] Finished trial 300 / 300. Current best value is 0.24269605823130108 with parameters: {'_min': 0.004028593139679666, 'mx_depth': 20.0}.


In [29]:
bests = load_pkl('tuned_parameters/decison_tree2.pkl')
bests.append(study4)

In [31]:
save_pkl('tuned_parameters/decision_tree.pkl', bests)

In [32]:
print('Best Parameter without Any:\n', bests[0].best_params)
print('Best Parameter without Any:\n', bests[1].best_params)

Best Parameter without Any:
 {'mx_depth': 20.0}
Best Parameter without Any:
 {'_min': 0.004028593139679666, 'mx_depth': 20.0}


In [8]:
a = load_pkl('tuned_parameters/decison_tree2.pkl')
a[0].best_params

{'mx_depth': 20.0}

In [13]:
best = {'_min': 0.003941570792418628, '_max': 0.7, 'ngram': (1, 2), 'mx_depth': 20.0, 
 'mn_sp': 0.17573596009797782, 'mn_lf': 1, 'mx_feat': 0.8376154434188189}
save_pkl('tuned_parameters/decision_tree.pkl', best)

In [11]:
best_class_weight = {'_min': 0.004, '_max': 0.75, 'ngram': (1, 2), 'mx_depth': 20, 'mn_sp': 0.18, 'mn_lf': 1, 'mx_feat': 0.85}
save_pkl('tuned_parameters/decision_tree_class_weight.pkl', best_class_weight)